## Код для предобработки текстов приговоров.

Чтобы импортировать функции из этой тетрадки(и из любой другой), нужно установить import-ipynb:

In [10]:
!pip install import-ipynb

  Running setup.py bdist_wheel for import-ipynb ... done
  Stored in directory: /Users/vera/Library/Caches/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Затем важно, чтобы все нужные тетрадки лежали в одной директории. Потом запускаем такое: 

In [11]:
import import_ipynb
import meta_extraction


importing Jupyter notebook from meta_extraction.ipynb


Готово! Теперь функции из этой тетрадки доступны в вашей тетрадке. Вы восхитительны!

Приговоры в формате xml, чтобы работать с данными, нам нужно их распарсить.

In [12]:
import os
import re
import nltk
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
from nltk.collocations import *
from nltk.corpus import stopwords
from nltk.util import everygrams
from collections import Counter, OrderedDict
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer


In [5]:
def parse_xml(path):   # парсим xml
    
    meta_data = {'court': '', 'judge': '', 'prosecutor': '', 'secretary': '', 'accused': '', 'result': '', 'category': '', 
            'punishment_type': '', 'punishment_term': ''} #складываем мета-данные в словарь
    
    with open(path, 'r', encoding='utf-8')as f:
        
        soup = BeautifulSoup(f.read(), 'html.parser')
        
        meta_data['court'] = soup.court.string     # добавляем в метаданные то, что имеется в разметке
        meta_data['judge'] = soup.judge.string
        meta_data['result'] = soup.result.string
        meta_data['category'] = soup.category.string
        
        html = []
        for line in soup.body:
            line = line.replace('[', '')
            line = line.replace(']', '') 
            html.append(line)
        true_html = ' '.join(html)
                
        html_soup = BeautifulSoup(true_html, 'html.parser')
        
        return html_soup.get_text(), meta_data
    
    
def get_parts(text):    # делим на части
    
    lines = [line for line in text.split('\n')]
    beg, end = 0, 0 
    
    for num, line in enumerate(lines):
        
        if re.search(r'у\s*с\s*т\s*а\s*н\s*о\s*в\s*и\s*л', line.lower()):
            beg = num + 1
            
        if re.search(r'п\s*р\s*и\s*г\s*о\s*в\s*о\s*р\s*и\s*л', line.lower()) \
            or re.search(r'п\s*о\s*с\s*т\s*а\s*н\s*о\s*в\s*и\s*л', line.lower()):
            
            end = num + 1
            
        if beg and end:
            
            beginning = [stroka.strip() for stroka in lines[:beg]]
            main_part = [stroka.strip() for stroka in lines[beg:end]]
            ending = [stroka.strip() for stroka in lines[end:]]
            
            return [' '.join(main_part), ' '.join(beginning), ' '.join(ending)]
        
        
def clean(text):
    
    pattern1 = re.compile(r'(п|ч|ст)(\.|\s|\d)')
    pattern2 = re.compile(r'((У|Г)П?К|КоАП|ПДД)\sРФ')
    pattern3 = re.compile('\d')
    pattern4 = re.compile('(ДД.ММ.ГГ|дд.мм.гг)')

    text = re.sub(pattern1, ' ABBR ', text)
    text = re.sub(pattern2, ' DOCUMENT ', text)
    text = re.sub(pattern3, ' DIGIT ', text)
    text = re.sub(pattern4, ' DATE ', text)
    text = text.replace('\xad', '')
    text = text.replace('п р и г о в о р и л', '')
    text = text.replace('п о с т а н о в и л', '')
    text = text.replace('П Р И Г О В О Р И Л', '')
    text = text.replace('П О С Т А Н О В И Л', '')
    
    return text
        
    
def process_xml(path):
    text, meta_data = parse_xml(path)
    main_part = get_parts(text)[0]
    cleaned_main_part = clean(main_part)
    try:
        return cleaned_main_part, meta_data
    except TypeError:
        return None

In [8]:
def normalize(text):
    
    stops = set(stopwords.words('russian'))
    punct = punctuation+'«»—…“”*№–'
    morph = MorphAnalyzer()
    words = [word.strip(punct) for word in text.lower().split()] 
    words = [word for word in words if word]
    words = [morph.parse(word)[0].normal_form for word in words if word]

    return words

In [13]:
# Убрать в отдельный модуль с заранее подготовленной train_data, и чтобы принимал только text
def segmentate(train_data:str, text: str): #очень желательно, чтобы тренировочная выборка отличалась от той, на которую ее применяют.
    
    trainer = PunktTrainer()
    trainer.INCLUDE_ALL_COLLOCS = True
    trainer.train(train_data) #на случай, если тренировочные данные в формате списка, добавить ('\n'.join(train_data))
    tokenizer = PunktSentenceTokenizer(trainer.get_params())
    
    return tokenizer.tokenize(text)

In [ ]:
def preprocess(path):
    text, meta_data = process_xml(path)
    normalized_text = normalize(text)
    
    return normalized_text, meta_data